In [1]:
import numpy as np
from sklearn.datasets import make_regression
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [2]:
X, y = make_regression(10000, 10, noise=20)

In [3]:
class RidgeReg(LinearBase):
    
    """Ridge Regression"""
    
    def __init__(self, learning_rate, n_iter, alpha):
        super().__init__(learning_rate, n_iter)
        self.alpha = alpha

    def fit(self, X, y, optimization='normal', verbose=None):
        self.input_dim = X.shape[1]
        
        # add 1s
        xb = np.c_[np.ones((X.shape[0],1)), X]
        
        # initiate coefs
        theta = np.ones((xb.shape[1], 1))
        
        # get m
        m = xb.shape[0]
        
        # normal equation
        if optimization=='normal':
            if m < 50000:
                
                # identity matrix
                id_m = np.eye(len(theta))
                id_m[0][0] = 0
                A = self.alpha * id_m  
                
                # coefficients
                theta = np.linalg.inv(xb.T.dot(xb) + A).dot(xb.T).dot(y)
            else:
                optimization = 'gradient_descent'
                
        if optimization=='gradient_descent':
            for step in range(self.n_iter):
                
                super().fit(xb, y)
                
                error = self._error(xb, y, theta)

                 # update gradients 
                gradients = 2 / self.m * xb.T.dot(error) + (2 * self.alpha * theta)

                # update coefficients            
                theta -= self.alpha * gradients
        
        self.coefs = theta[1:]
        self.intercept = theta[0]
        self.fitted = True
        return self
   

NameError: name 'LinearBase' is not defined

In [4]:
class LinearBase(object):
    
    """Base class for linear models"""
    
    def __init__(self, learning_rate, n_iter):
        self.learning_rate = learning_rate
        self.n_iter = n_iter

    def fit(self, X, y):
        """
        Fit using gradient descent.
        """
        for i in range(self.n_iter):

            error = self._error(X, y, self.theta)

            # update gradients 
            gradients = 2 / self.m * X.T.dot(error)
        
            # update theta 
            self.theta -= self.learning_rate * gradients
            
    def _error(self, X, y, theta):
        return X.dot(theta) - y.reshape(-1,1)

    def predict(self, X):
        xm, xn = X.shape
        
        if not self.fitted:
            raise Exception(f"{__class__.__name__} is not yet fitted.")
            
        if len(X.shape) == 1:
            X = X.reshape(1,-1)
            
        if xn != self.n:
            raise Exception(f"Input data shape must be equal to fit data shape {self.n}")
            
        if xn == 1:
            return X.dot(self.coefs) + self.intercept
        
        return X.dot(self.coefs)
    
    def __repr__(self):
        if self.fitted:
            return f"coefficients: {self.coefs}, \n\n intercept: {self.intercept}"
        return self.__class__.__name__

In [113]:

class LassoReg(LinearBase):
    
    """LASSO Regression using coordinate gradient descent"""
    
    def __init__(self, n_iter=2000, alpha=1):
        super().__init__(learning_rate=None, n_iter=1000)
        self.alpha = alpha
    
    def fit(self, X, y):
        self.m, self.n = X.shape
        
        # add 1s for intercept
        xb = np.hstack((np.ones((self.m, 1)), X))
            
        # initialise theta    
        theta = np.zeros(xb.shape[1])

        # set intercept
        theta[0] = np.sum(y - np.dot(xb[:, 1:], theta[1:])) / self.m
        
        for step in range(self.n_iter):
            
            for param in range(1, self.n):
                
                # temporary theta
                theta_ = theta
                theta_[param] = 0.0
                
                # residuals
                err = y - np.dot(xb, theta_)
                
                # input to thresholding
                x = np.dot(xb[:, param], err)
                lmbda = self.alpha * self.m

                # update coefficient
                theta[param] = self._soft_thresholding(x, lmbda) / (xb[:, param] ** 2).sum()

                # set intercept
                theta[0] = np.sum(y - np.dot(xb[:, 1:], theta[1:])) / self.m

    
        self.intercept = theta[0].flatten()
        self.coefs = theta[1:].flatten()
        self.fitted = True
        return self

    @staticmethod
    def _soft_thresholding(x, lmbda):
        if x > 0 and lmbda < abs(x):
            return x - lmbda
        elif x < 0 and lmbda < abs(x):
            return x + lmbda
        return 0
   

### LASSO regression

In [114]:
from sklearn.linear_model import Lasso

In [115]:
X, y = make_regression(1000, 20, noise=20, n_informative=5)

In [116]:
las = Lasso(alpha=.5, ).fit(X, y)
las.coef_

array([ 0.00000000e+00, -1.51932394e-02,  8.83123564e-01, -0.00000000e+00,
        4.27571463e-01, -0.00000000e+00,  7.06020276e+01, -2.16362444e-02,
        6.83126342e-01,  4.08738379e+01,  8.43117818e+01, -0.00000000e+00,
        8.70087270e+01, -2.72782849e-01,  0.00000000e+00,  6.59254854e+01,
        0.00000000e+00, -0.00000000e+00,  1.48672026e-01,  0.00000000e+00])

In [117]:
lasso_preds = np.ones(20)

In [118]:
las.predict(lasso_preds.reshape(1,-1))

array([350.53331548])

In [119]:
las_sct = LassoReg(alpha=.5).fit(X, y)
las_sct.coefs

array([ 0.00000000e+00, -1.51933347e-02,  8.83175608e-01,  0.00000000e+00,
        4.27603365e-01,  0.00000000e+00,  7.06019977e+01, -2.16191739e-02,
        6.83141256e-01,  4.08738305e+01,  8.43117835e+01,  0.00000000e+00,
        8.70087314e+01, -2.72787620e-01,  0.00000000e+00,  6.59254885e+01,
        0.00000000e+00,  0.00000000e+00,  1.48672118e-01,  0.00000000e+00])

In [120]:
las_sct.predict(lasso_preds.reshape(1,-1))

array([350.5548238])

# Test vs sklearn

### Ridge Regression

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
m = Ridge(alpha=0.5).fit(X, y)


In [ ]:
m.coef_, m.intercept_

In [ ]:
q = RidgeReg(learning_rate=0.1, n_iter = 1000, alpha = 0.5)
q.fit(X, y)


In [ ]:
predictors = np.ones(10)

In [ ]:
q.predict(predictors)

In [ ]:
m.predict(predictors.reshape(1, -1))